<a href="https://colab.research.google.com/github/heber-augusto/sus-kpis-analysis/blob/main/sia/teste_agrega%C3%A7%C3%A3o_sus_ibge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook de teste de agregação entre dados das bases do SUS e IBGE

# Configurações iniciais para conectar com bucket no Google Storage
  - Definição do nome do projeto/bucket/pastas (local e bucket)

In [1]:
# id do projeto
project_id = 'teak-ellipse-317015'
# id do bucket dentro do projeto
bucket_id = 'observatorio-oncologia'

# nome da pasta local para mapear
local_folder_name = 'colab'

# nome da pasta do projeto
project_folder_name = 'monitor'

!gcloud config set project {project_id}

Updated property [core/project].


# Instalação para garantir montagem da pasta no bucket
Instalação do gcsfuse para mapear pasta do bucket no google colab

In [2]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  66475      0 --:--:-- --:--:-- --:--:-- 64853
OK
15 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 14.0 MB of archives.
After this operation, 31.2 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 123105 files and directories currently installed.)
Preparing to unpack .../gcsfuse_1.0.0_amd64.deb ...
Unpacking gcsfuse (1.0.0) ...
Setting up gcsfuse (1.0.0) ...


In [5]:
service_account_json = '/content/gcp-leitura.json'

# Montagem do bucket em uma pasta local do google colab

In [6]:
!mkdir {local_folder_name}
!gcsfuse --key-file {service_account_json} --implicit-dirs {bucket_id} {local_folder_name}

mkdir: cannot create directory ‘colab’: File exists
I0717 01:40:40.159740 2023/07/17 01:40:40.159705 Start gcsfuse/1.0.0 (Go version go1.20.4) for app "" using mount point: /content/colab


# Leitura de dados agregados por cidade (bases SIA PA, AQ e AR)

In [21]:
import pandas as pd


dados_municipios = pd.read_parquet('/content/colab/monitor/SP/consolidado/dados_estad_municipio_mensal.parquet.gzip')
dados_municipios.municipio = dados_municipios.municipio.astype('int')
dados_municipios

,data,municipio,primeiro_estadiamento,custo_estadiamento,numero_pacientes,obtitos,obito_futuro,numero_procedimentos,numero_diagnosticos
0,200801,110002,1,213.40,1,0,0,1,1.0
1,200801,110002,2,2458.65,2,0,0,2,2.0
2,200801,110004,1,79.75,1,0,0,1,1.0
3,200801,110004,2,239.25,3,0,0,3,3.0
4,200801,110004,3,4837.55,3,0,1,3,3.0
...,...,...,...,...,...,...,...,...,...
492793,202304,530010,3,159.50,3,0,0,3,0.0
492794,202304,530010,4,651.25,3,0,0,3,0.0
492795,202304,530040,1,0.00,1,0,0,1,0.0
492796,202304,530140,3,79.75,1,0,0,1,0.0


# Dados do IBGE

 - Códigos e nomes de municipios (ibge_cidades.csv)
 - Dados demográficos (ibge_pop_sexo_grupos_idade_municipios.csv)

In [23]:
codigo_municipios = pd.read_csv(
    '/content/colab/monitor/ibge_data/ibge_cidades.csv',
    sep = ';')

codigo_municipios.loc[:,'municipio'] = (codigo_municipios.id / 10).astype('int')

codigo_municipios.columns = ['id_ibge', 'cidade', 'id_uf', 'estado', 'municipio']
codigo_municipios

,id_ibge,cidade,id_uf,estado,municipio
0,1100015,Alta Floresta D'Oeste,11,Rondônia,110001
1,1100023,Ariquemes,11,Rondônia,110002
2,1100031,Cabixi,11,Rondônia,110003
3,1100049,Cacoal,11,Rondônia,110004
4,1100056,Cerejeiras,11,Rondônia,110005
...,...,...,...,...,...
5565,5222005,Vianópolis,52,Goiás,522200
5566,5222054,Vicentinópolis,52,Goiás,522205
5567,5222203,Vila Boa,52,Goiás,522220
5568,5222302,Vila Propício,52,Goiás,522230


In [17]:
dados_demograficos = pd.read_csv(
    '/content/colab/monitor/ibge_data/ibge_pop_sexo_grupos_idade_municipios.csv',
    sep = ';')



dados_demograficos.loc[:,'municipio'] = (dados_demograficos.codigo_local / 10).astype('int')
dados_demograficos

,codigo_sexo,valor_sexo,codigo_local,nome_local,ano,populacao,faixa_idade,municipio
0,4,Homens,1100015,Alta Floresta D'Oeste - RO,2010,12656,Total,110001
1,4,Homens,1100023,Ariquemes - RO,2010,45543,Total,110002
2,4,Homens,1100031,Cabixi - RO,2010,3266,Total,110003
3,4,Homens,1100049,Cacoal - RO,2010,39124,Total,110004
4,4,Homens,1100056,Cerejeiras - RO,2010,8551,Total,110005
...,...,...,...,...,...,...,...,...
556495,5,Mulheres,5222005,Vianópolis - GO,2010,...,Total,522200
556496,5,Mulheres,5222054,Vicentinópolis - GO,2010,...,Total,522205
556497,5,Mulheres,5222203,Vila Boa - GO,2010,...,Total,522220
556498,5,Mulheres,5222302,Vila Propício - GO,2010,...,Total,522230


In [19]:
dados_demograficos[dados_demograficos.codigo_local == 1100015]

,codigo_sexo,valor_sexo,codigo_local,nome_local,ano,populacao,faixa_idade,municipio
0,4,Homens,1100015,Alta Floresta D'Oeste - RO,2010,12656,Total,110001
5565,5,Mulheres,1100015,Alta Floresta D'Oeste - RO,2010,11736,Total,110001
11130,4,Homens,1100015,Alta Floresta D'Oeste - RO,2010,928,Total,110001
16695,5,Mulheres,1100015,Alta Floresta D'Oeste - RO,2010,923,Total,110001
22260,4,Homens,1100015,Alta Floresta D'Oeste - RO,2010,168,Total,110001
...,...,...,...,...,...,...,...,...
528675,5,Mulheres,1100015,Alta Floresta D'Oeste - RO,2010,-,Total,110001
534240,4,Homens,1100015,Alta Floresta D'Oeste - RO,2010,-,Total,110001
539805,5,Mulheres,1100015,Alta Floresta D'Oeste - RO,2010,-,Total,110001
545370,4,Homens,1100015,Alta Floresta D'Oeste - RO,2010,...,Total,110001


# Dados por município com nome do município

In [24]:
dados_municipios.merge(
    codigo_municipios[['municipio','cidade', 'estado']],
    how='left',
    on='municipio'
)

,data,municipio,primeiro_estadiamento,custo_estadiamento,numero_pacientes,obtitos,obito_futuro,numero_procedimentos,numero_diagnosticos,cidade,estado
0,200801,110002,1,213.40,1,0,0,1,1.0,Ariquemes,Rondônia
1,200801,110002,2,2458.65,2,0,0,2,2.0,Ariquemes,Rondônia
2,200801,110004,1,79.75,1,0,0,1,1.0,Cacoal,Rondônia
3,200801,110004,2,239.25,3,0,0,3,3.0,Cacoal,Rondônia
4,200801,110004,3,4837.55,3,0,1,3,3.0,Cacoal,Rondônia
...,...,...,...,...,...,...,...,...,...,...,...
492793,202304,530010,3,159.50,3,0,0,3,0.0,Brasília,Distrito Federal
492794,202304,530010,4,651.25,3,0,0,3,0.0,Brasília,Distrito Federal
492795,202304,530040,1,0.00,1,0,0,1,0.0,NaN,NaN
492796,202304,530140,3,79.75,1,0,0,1,0.0,NaN,NaN
